<a href="https://colab.research.google.com/github/Jordaf/game/blob/main/752_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -O /content/train_part2.npz https://image-net.org/data/downsample/Imagenet64_train_part2.zip

--2024-12-14 01:35:03--  https://image-net.org/data/downsample/Imagenet64_train_part2.zip
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6834457024 (6.4G) [application/zip]
Saving to: ‘/content/train_part2.npz’

/content/train_part 100%[===================>]   6.36G  2.47MB/s    in 44m 7s  

2024-12-14 02:19:11 (2.46 MB/s) - ‘/content/train_part2.npz’ saved [6834457024/6834457024]



In [3]:
!wget -O /content/val_data.npz https://image-net.org/data/downsample/Imagenet64_val.zip

--2024-12-14 02:19:11--  https://image-net.org/data/downsample/Imagenet64_val.zip
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533691161 (509M) [application/zip]
Saving to: ‘/content/val_data.npz’

/content/val_data.n 100%[===================>] 508.97M  5.24MB/s    in 1m 44s  

2024-12-14 02:20:55 (4.89 MB/s) - ‘/content/val_data.npz’ saved [533691161/533691161]



In [ ]:
!wget -O /content/train_part1.npz https://image-net.org/data/downsample/Imagenet64_train_part1.zip

--2024-12-14 02:20:55--  https://image-net.org/data/downsample/Imagenet64_train_part1.zip
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6837429926 (6.4G) [application/zip]
Saving to: ‘/content/train_part1.npz’

/content/train_part  73%[=============>      ]   4.70G   365KB/s    eta 23m 24s

In [ ]:
import os

file_path = '/content/val_data.npz'
print(f"Validation file size: {os.path.getsize(file_path)} bytes")


Validation file size: 533691161 bytes


In [ ]:
import os

file_path = '/content/train_part1.npz'
print(f"train1 file size: {os.path.getsize(file_path)} bytes")

train1 file size: 6837429926 bytes


In [ ]:
import os

file_path = '/content/train_part2.npz'
print(f"train2 file size: {os.path.getsize(file_path)} bytes")

train2 file size: 6834457024 bytes


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pickle
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class NPZDataset(Dataset):
    def __init__(self, npz_files, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform

        for file in npz_files:
            data = np.load(file, allow_pickle=True)
            for key in data.keys():
                batch = pickle.loads(data[key])
                self.images.append(batch['data'])
                self.labels.append(batch['labels'])

        self.images = np.concatenate(self.images, axis=0)
        self.labels = np.concatenate(self.labels, axis=0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].reshape(3, 64, 64) / 255.0
        label = self.labels[idx] - 1
        if self.transform:
            image = self.transform(torch.tensor(image, dtype=torch.float32))
        return image, label

train_files = ['/content/train_part1.npz', '/content/train_part2.npz']
val_file = '/content/val_data.npz'

train_dataset = NPZDataset(train_files, transform=transform)
val_dataset = NPZDataset([val_file], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of evaluation samples: {len(val_dataset)}")

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

num_classes = 50
model = SimpleCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train_and_evaluate_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_losses.append(running_loss / len(train_loader))
        train_accuracies.append(correct / total)

        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                outputs = outputs[:labels.size(0)]
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(correct / total)

        print(f"Epoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}, "
              f"Evaluation Loss: {val_losses[-1]:.4f}, Evaluation Accuracy: {val_accuracies[-1]:.4f}")

    return train_losses, train_accuracies, val_losses, val_accuracies

train_losses, train_accuracies, val_losses, val_accuracies = train_and_evaluate_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10
)

FileNotFoundError: [Errno 2] No such file or directory: '/content/train_part1.npz'

In [ ]:
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channel, channel // reduction, bias=False)
        self.fc2 = nn.Linear(channel // reduction, channel, bias=False)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch, channel, _, _ = x.size()
        y = x.mean(dim=[2, 3])
        y = self.relu(self.fc1(y))
        y = self.sigmoid(self.fc2(y))
        y = y.view(batch, channel, 1, 1)
        return x * y

class DualStreamDataset(Dataset):
    def __init__(self, npz_files, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform

        for file in npz_files:
            data = np.load(file, allow_pickle=True)
            for key in data.keys():
                batch = pickle.loads(data[key])
                self.images.append(batch['data'])
                self.labels.append(batch['labels'])

        self.images = np.concatenate(self.images, axis=0)
        self.labels = np.concatenate(self.labels, axis=0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].reshape(3, 64, 64) / 255.0
        label = self.labels[idx] - 1
        if self.transform:
            original = self.transform(torch.tensor(image, dtype=torch.float32))
            augmented = self.transform(torch.tensor(image, dtype=torch.float32))
        return original, augmented, label

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_files = ['/content/train_part1.npz', '/content/train_part2.npz']
val_file = '/content/val_data.npz'

train_dataset = DualStreamDataset(train_files, transform=transform)
val_dataset = DualStreamDataset([val_file], transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

num_classes = len(np.unique(train_dataset.labels))
print(f"Number of classes: {num_classes}")

class DualStreamCNN(nn.Module):
    def __init__(self, num_classes):
        super(DualStreamCNN, self).__init__()
        self.stream1_conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.stream1_conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.stream1_conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.stream1_se = SEBlock(128)

        self.stream2_conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.stream2_conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.stream2_conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.stream2_se = SEBlock(128)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 4 * 4 * 2, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x1, x2):
        x1 = self.pool(F.relu(self.stream1_conv1(x1)))
        x1 = self.pool(F.relu(self.stream1_conv2(x1)))
        x1 = self.pool(F.relu(self.stream1_conv3(x1)))
        x1 = self.stream1_se(x1)

        x2 = self.pool(F.relu(self.stream2_conv1(x2)))
        x2 = self.pool(F.relu(self.stream2_conv2(x2)))
        x2 = self.pool(F.relu(self.stream2_conv3(x2)))
        x2 = self.stream2_se(x2)

        x1 = x1.view(-1, 128 * 4 * 4)
        x2 = x2.view(-1, 128 * 4 * 4)
        fused = torch.cat((x1, x2), dim=1)

        fused = F.relu(self.fc1(fused))
        output = self.fc2(fused)
        return output

model = DualStreamCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train_dual_stream_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs1, inputs2, labels in train_loader:
            inputs1, inputs2, labels = inputs1.to(device), inputs2.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs1, inputs2)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

def evaluate_dual_stream_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs1, inputs2, labels in val_loader:
            inputs1, inputs2, labels = inputs1.to(device), inputs2.to(device), labels.to(device)
            outputs = model(inputs1, inputs2)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")
    return accuracy

train_dual_stream_model(model, train_loader, criterion, optimizer, num_epochs=5)
evaluate_dual_stream_model(model, val_loader)